# 🗑️ ПОЛНАЯ VLM для мусора

**CV часть = ваши модели:**
1. Ансамбль детекторов (YOLO + RT-DETR) → мусор
2. Классификатор сцен (MobileNet) → grass, marshy, rocky, sandy

**Результат:** "There is 2 plastic on the grass"


In [ ]:
# 1. Установка
!pip install -q roboflow ultralytics supervision transformers pillow tqdm
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 2. Скачать датасет мусора
!python download_roboflow_dataset.py


In [ ]:
# 3. Скачать датасет сцен (Roboflow Terrain Classification)
!python download_scene_dataset.py --download


In [ ]:
# 4. Обучение классификатора сцен (~5-10 мин на GPU)
!python train_scene_classifier.py --train_dir data/scene_dataset/train --val_dir data/scene_dataset/val --output models/scene_classifier.pt --epochs 15 --batch_size 32


In [ ]:
# 5. Загрузка полной VLM
from VLM_Complete import CompleteVLM
from pathlib import Path
import random, matplotlib.pyplot as plt, matplotlib.patches as patches
from PIL import Image

vlm = CompleteVLM(
    yolo_path="models/yolo/yolov8x/best.pt",
    detr_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    scene_classifier_path="models/scene_classifier.pt",
    conf_threshold=0.5
)


In [ ]:
# 6. Тестирование
valid = Path("data/roboflow_dataset/valid")
test = Path("data/roboflow_dataset/test")
images = list(valid.glob("*.jpg")) + list(test.glob("*.jpg"))
print(f"✅ {len(images)} изображений")

COLORS = {'glass':'green', 'plastic':'red', 'metal':'cyan', 'paper':'yellow', 'organic':'lime'}

def test_image(img_path):
    img = Image.open(img_path).convert('RGB')
    analysis = vlm.get_full_analysis(img)
    fig, ax = plt.subplots(figsize=(10,7))
    ax.imshow(img)
    for d in analysis['garbage']['detections']:
        b = d['box']
        c = COLORS.get(d['label'], 'white')
        ax.add_patch(patches.Rectangle((b[0],b[1]), b[2]-b[0], b[3]-b[1], lw=2, ec=c, fc='none'))
        ax.text(b[0], b[1]-5, f"{d['label']} {d['confidence']:.0%}", color=c, fontsize=9, weight='bold')
    scene = analysis['scene']
    title = f"Scene: {scene['class']} ({scene['confidence']:.0%})" if scene['confidence'] >= 0.8 else "Scene: uncertain"
    ax.set_title(f"{title} | Objects: {analysis['garbage']['total']}")
    ax.axis('off')
    plt.show()
    print(f"📝 {analysis['description']}")


In [ ]:
# 7. Тест на 3 изображениях
for img in random.sample(images, min(3, len(images))):
    print(f"\n{'='*60}\n📷 {img.name}")
    test_image(img)


In [ ]:
# 8. Интерактивные вопросы
img = random.choice(images)
test_image(img)
for q in ["What is here?", "Is there plastic?", "How many objects?", "Where is the garbage?"]:
    print(f"❓ {q}\n✅ {vlm.answer(str(img), q)}\n")


In [ ]:
# 9. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/VLM_garbage/models
!cp models/scene_classifier.pt /content/drive/MyDrive/VLM_garbage/models/
!cp -r models/yolo models/rt-detr /content/drive/MyDrive/VLM_garbage/models/ 2>/dev/null || true
!cp VLM_Complete.py train_scene_classifier.py /content/drive/MyDrive/VLM_garbage/
print("✅ Сохранено!")
